In [ ]:
import sys
sys.path.append('../')

In [ ]:
%load_ext autoreload

In [ ]:
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)
import numpy as np
import matplotlib.pyplot as plt
import ceedub as cw

from scipy.signal import argrelmax
from scipy.optimize import minimize_scalar

from ecc_prior.ecc_burst import EccBurst

%matplotlib inline
%autoreload 2

# Definitions!

In [ ]:
GMsun = 1.32712440018e20  # m^3/s^2
c = 299792458 # m/s

Rsun = GMsun / c**2
Tsun = GMsun / c**3

In [ ]:
def bigauss(x, x0=np.array([0,0]), cov=np.diag([1,1])):
    """bivariate Gaussian
    """
    x = np.asarray(x)
    x0 = np.asarray(x0)
    cov = np.asarray(cov)
    icov = np.linalg.inv(cov)
    dx = x-x0
    
    #norm = 1/np.sqrt(2*np.pi * np.linalg.det(cov))
    arg = -0.5 * np.einsum('i,ij,j', dx, icov, dx)
    return np.exp(arg)  # don't normalize
    #return norm * np.exp(arg)

# read in data

this data was generated with $q = 0.25$

In [ ]:
t_dat = np.loadtxt('t.dat')
hp_dat = np.loadtxt('hp.dat')
hc_dat = np.loadtxt('hc.dat')

A_dat = np.sqrt(hp_dat**2 + hc_dat**2)  # intrinsic GW amplitude

In [ ]:
plt.plot(t_dat, hp_dat)
plt.xlabel('$t/M$');

In [ ]:
plt.plot(t_dat, A_dat)
plt.xlabel('$t/M$');
plt.xlim([33000, 37500]);

In [ ]:
# set t=0 to merger
i_merge = np.argmax(hp_dat**2 + hc_dat**2)
t_merge = t_dat[i_merge]
t_dat -= t_merge

the "M" naming convention marks quantities in units of total mass, M.

In [ ]:
# assume optimimally oriented +-polarization

# get subset for faster CWT and plotting...
i_start = 34000
tM = t_dat[i_start:]
h = hp_dat[i_start:]
A = A_dat[i_start:]

dtM = tM[1] - tM[0]
N = len(tM)

In [ ]:
# find local maxima (t)
ii_local_max = argrelmax(A)[0]

tM[ii_local_max]

# wavelet transform data

In [ ]:
# wavelet transform subset of data
dJ = 1/16

WB = cw.WaveletBasis(wavelet=cw.MorletWave(), N=N, dt=dtM, dj=dJ)
fM = WB.freqs

wdat = WB.cwt(h)
wpow = np.real(wdat*wdat.conj())

In [ ]:
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111)
ax.pcolormesh(tM, fM, wpow, cmap='YlOrRd')

ax.set_xlabel('$t/M$', fontsize=20);
ax.set_ylabel('$fM$', fontsize=20);
ax.tick_params(labelsize=16)

ax.set_yscale('log')

In [ ]:
# find local maxima for f
jj_local_max = np.argmax(wpow[:,ii_local_max], axis=0)

tf_correct = [[tM[ii], fM[jj]] for ii,jj in zip(ii_local_max, jj_local_max)]

these are the **correct** burst locations

In [ ]:
ts, fs = np.array(tf_correct).T

fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111)
ax.pcolormesh(tM, fM, wpow, cmap='Blues')
ax.scatter(ts, fs, marker='x', s=100, c='r')

ax.set_xlabel('$t/M$', fontsize=20);
ax.set_ylabel('$fM$', fontsize=20);
ax.tick_params(labelsize=16)

ax.set_yscale('log')

# Prior Stuff

In [ ]:
Mtot = 30  # total mass
q = 0.25  # mass ratio

eb = EccBurst(q)  # works in units of total mass, completely determined by mass ratio!

## get meta params $t_*, f_*, r_{p*}, \delta e_*$

In [ ]:
i_bright = 2  # the pre-merger burst (brightest!?)
tstar, fstar = tf_correct[i_bright]  

rpstar = (2*np.pi*fstar)**(-2/3) # periastron of burst via Kepler (-> * M^(1/3) = 1)

# find eccentricity for this burst
tprev, fprev = tf_correct[i_bright-1]
tnext, fnext = tf_correct[i_bright+1]


# also check fprev/fnext? rewrite as lambda statement?
def diff_back_t(de):
    """find ecc that minimizes the difference"""
    rpstar =  ((2-de)/(2*np.pi*fstar)**2)**(1/3)
    t, f = eb.tf_backward(tstar, fstar, rpstar, de, re=False)
    return np.abs(t - tprev)

def diff_for_t(de):
    """find ecc that minimizes the difference"""
    rpstar =  ((2-de)/(2*np.pi*fstar)**2)**(1/3)
    t, f = eb.tf_forward(tstar, fstar, rpstar, de, re=False)
    return np.abs(t - tnext)

result = minimize_scalar(diff_back_t, bracket=(0.1, 0.4, 0.8), tol=1e-6)
de_back_t = result.x

result = minimize_scalar(diff_for_t, bracket=(0.1, 0.4, 0.8), tol=1e-6)
de_for_t = result.x

rp_back = ((2-de_back_t)/(2*np.pi*fstar)**2)**(1/3)
rp_for = ((2-de_for_t)/(2*np.pi*fstar)**2)**(1/3)

print(de_back_t, de_for_t)
print(rp_back, rp_for)

In [ ]:
tf_correct

In [ ]:
destar = de_back_t # (de_for_t + de_back_t) / 2
rpstar = ((2-destar)/(2*np.pi*fstar)**2)**(1/3)

### these are the meta params (in units of total mass!)

In [ ]:
print("Mc = {:.4f}".format(eb.Mchirp))
print(" t = {:.4f}".format(tstar))
print(" f = {:.4f}".format(fstar))
print("rp = {:.4f}".format(rpstar))
print("de = {:.4f}".format(destar))

In [ ]:
prior_bursts = eb.get_all_bursts(tstar, fstar, rpstar, destar, tmin=tM[0], tmax=tM[-1])

these are **where prior** puts blobs

In [ ]:
ts, fs = np.array(prior_bursts).T

fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111)
ax.pcolormesh(tM, fM, wpow, cmap='Blues')
ax.scatter(ts, fs, marker='x', s=100, c='r')

ax.set_xlabel('$t/M$', fontsize=20);
ax.set_ylabel('$fM$', fontsize=20);
ax.tick_params(labelsize=16)

ax.set_yscale('log')

## Convert to SI units

In [ ]:
M2sec = Mtot * Tsun
ts = tM * M2sec
fs = fM / M2sec

tf_prior_SI = [[t*M2sec, f/M2sec] for t,f in prior_bursts]
tf_correct_SI = [[t*M2sec, f/M2sec] for t,f in tf_correct]

these are the meta paramters in usual units!

In [ ]:
print(" M = {:.3f} Msun".format(Mtot))
print("Mc = {:.3f} Msun".format(eb.Mchirp*Mtot))
print(" t = {:.3f} sec".format(tstar*M2sec))
print(" f = {:.3f} Hz".format(fstar/M2sec))
print("rp = {:.3f} Mtot".format(rpstar))
print("de = {:.3f}".format(destar))

## plot data

In [ ]:
time_box = [0.15, 0.75, 0.8, 0.2] # left, bottom, width, height
wave_box = [0.15, 0.15, 0.8, 0.6]

In [ ]:
fig = plt.figure(figsize=(8,6))
axt = fig.add_axes(time_box)
axt.plot(ts, h/max(h))
axt.set_xlim([ts[0],ts[-1]])
axt.set_xticklabels([])
axt.set_yticklabels([])

axw = fig.add_axes(wave_box)
axw.pcolormesh(ts, fs, wpow, cmap='Blues')
axw.set_ylim([10, 1000]);
axw.set_xlabel('$t$ (sec)', fontsize=20);
axw.set_ylabel('$f$ (Hz)', fontsize=20);
axw.tick_params(labelsize=16)
axw.set_yscale('log')

## make prior probability map for "these" meta params

In [ ]:
delTs, delFs = np.diff(tf_prior_SI, axis=0).T  # actual diffs
sigTs = np.hstack([delTs, [delTs[-1]*1.2]]) / 6  # scaled, reuse last entry
sigFs = np.hstack([delFs, [delFs[-1]*1.2]]) * 2
rhos = np.hstack([np.ones(len(delTs))*0.0, [0]])  # rho in +/-[0,1)

covs = [[[dT**2, dT*dF*rho], [dT*dF*rho, dF**2]]
            for dT,dF,rho in zip(sigTs, sigFs, rhos)]

def prior_test(t, f):
    prob = 0
    for (tx, fx), covx in zip(tf_prior_SI, covs):
        prob += bigauss([t,f], x0=[tx,fx], cov=covx)
    prob /= len(tf_prior_SI)
    return prob

In [ ]:
Nf = 100
Nt = 300

Fs = np.logspace(1, 3, Nf)
Ts = np.linspace(ts[0], ts[-1], Nt)

prior_map = np.zeros([Nf, Nt])

for ii,tt in enumerate(Ts):
    for jj,ff in enumerate(Fs):
        prior_map[jj,ii] = prior_test(tt, ff)

In [ ]:
t_actual, f_actual = np.array(tf_correct_SI).T

fig = plt.figure(figsize=(8,4.8))
ax = fig.add_subplot(111)
ax.pcolormesh(Ts, Fs, prior_map, cmap='Reds')
ax.scatter(t_actual, f_actual, marker='x', s=150, c='navy')
ax.set_ylim([10, 1000]);
ax.set_xlabel('$t$ (sec)', fontsize=20);
ax.set_ylabel('$f$ (Hz)', fontsize=20);
ax.tick_params(labelsize=16)
ax.set_yscale('log')

# Prior function should work like this

In [ ]:
tf_correct_SI  # this is the "data"

In [ ]:
tf_prior_SI

In [ ]:
def the_prior(tf_from_BW, Mtot, q, rpstar, destar, tstar, fstar):
    # enforce priors
    if q <= 0 or q > 1:
        return 0
    if destar < 1e-3 or destar > 0.9:
        return 0
    if rpstar < 2.5:
        return 0

    # recompute EccBurst w/ this q
    eb.q = q
    
    # get tstar, fstar in units of total mass
    M2sec = Mtot * Tsun
    tstar /= M2sec
    fstar *= M2sec

    # get bursts convert to SI w/ this Mtot
    prior_bursts = eb.get_all_bursts(tstar, fstar, rpstar, destar, tmin=tM[0], tmax=tM[-1])
    tf_prior_SI = [[t*M2sec, f/M2sec] for t,f in prior_bursts]

    sigTs, sigFs = np.array([[1/f, f/6] for t,f in tf_prior_SI]).T
    rhos = np.hstack([np.ones(len(delTs))*0.0, [0]])  # rho in +/-[0,1)

    covs = [[[dT**2, dT*dF*rho], [dT*dF*rho, dF**2]]
              for dT,dF,rho in zip(sigTs, sigFs, rhos)]

    prob = 1
    for t,f in tf_from_BW:
        this_prob = 0
        for (tx, fx), covx in zip(tf_prior_SI, covs):
            this_prob += bigauss([t,f], x0=[tx,fx], cov=covx)
        prob *= this_prob / len(tf_prior_SI)

    return prob

def log_prior(x, tf_data):
    return np.log(the_prior(tf_data, *x))

In [ ]:
np.log(the_prior(tf_correct_SI, *x_start))

In [ ]:
this_x = [34.8564248,  # Mtot
          0.2286404,  # q
          1.92864714,  # rpstar
          0.15210620,  # destar
          -0.003017523, # tstar
          191.14749]  # fstar
the_prior(tf_correct_SI, *this_x)

In [ ]:
nwalkers = 40

x_start = np.array([Mtot, q, rpstar, destar, tstar*M2sec, fstar/M2sec])
x_vars = np.array([5, 0.05, 1, 0.05, 0.002, 10])
ndim = len(x_start)

x0 = np.array([x_start + x_vars*np.random.randn(ndim) for i in range(nwalkers)])

In [ ]:
sampler = mc.EnsembleSampler(nwalkers, ndim, log_prior, args=[tf_correct_SI])

# burn in
pos, prob, state = sampler.run_mcmc(x0, 1000);
sampler.reset  # throw away

# restart from final position of burn
pos, prob, state = sampler.run_mcmc(pos, 10000);

In [ ]:
print("Mean acceptance fraction: {0:.3f}"
                .format(np.mean(sampler.acceptance_fraction)))

In [ ]:
params = [r"$M_{tot}$",
          r"$q$",
          r"$r_p$",
          r"$\delta e$",
          r"$t_\star$",
          r"$f_\star$",]

corner_kwargs = {'bins':30,
                 'labels':params,
                 'label_kwargs':{'fontsize':18},
                 'show_titles':True,
                 'smooth':0.5,
                 'plot_datapoints':False,
                 'plot_density':True,
                 'plot_contours':True,
                 'fill_contours':False,}

corner(sampler.flatchain, **corner_kwargs);

In [ ]:
fig = plt.figure(figsize=(12,4))

ax1 = fig.add_subplot(121)
_, bins, _ = ax1.hist(sampler.flatchain[:,4], bins=30)
ax2.set_xlabel(r"$t_\star (sec)$")
for t,f in tf_correct_SI:
    if t>bins[0] and t<bins[-1]:
        ax1.axvline(x=t, color='C1')

ax2 = fig.add_subplot(122)
_, bins, _ = ax2.hist(sampler.flatchain[:,5], bins=30)
ax2.set_xlabel(r"$f_\star (Hz)$")
for t,f in tf_correct_SI:
    if f>bins[0] and f<bins[-1]:
        ax2.axvline(x=f, color='C1')

In [ ]:
Mq_ch = sampler.flatchain[:,:2]
Mq_pars = [r"$M_{tot}$",
           r"$q$"]

truths = [Mtot, q]

corner_kwargs = {'bins':30,
                 'labels':Mq_pars,
                 'label_kwargs':{'fontsize':18},
                 'show_titles':True,
                 'truths':truths,
                 'smooth':0.5,
                 'plot_datapoints':False,
                 'plot_density':True,
                 'plot_contours':True,
                 'fill_contours':False,}

corner(Mq_ch, **corner_kwargs);

In [ ]:
def prior(t, f, Mtot, q, rpstar, destar, tstar, fstar):
    """compute prior for wavelet at (t,f)
    depends on physical metaparams...
    """
    #ts, fs, covs = tf_from_metaparams(Mtot, Mchirp, rpstar, destar, tstar, fstar)

    prob = 0
    for tx, fx, cx in zip(ts, fs, covs):
        prob += bigauss([t,f], x0=[tx,fx], cov=cx)
    prob /= len(ts)

    return prob

## generate prior maps from samples

In [ ]:
bounds = [-0.070, -0.050, -0.033, -0.009]
_, bins, _ = plt.hist(sampler.flatchain[:,4], bins=30)
ax2.set_xlabel(r"$t_\star (sec)$")
for t in bounds:
    if t>bins[0] and t<bins[-1]:
        plt.axvline(x=t, color='C1')

In [ ]:
cond = sampler.flatchain[:,4]<bounds[0]
subchain0 = sampler.flatchain[cond]

cond = np.bitwise_and(sampler.flatchain[:,4]>bounds[0], sampler.flatchain[:,4]<bounds[1])
subchain1 = sampler.flatchain[cond]

cond = np.bitwise_and(sampler.flatchain[:,4]>bounds[1], sampler.flatchain[:,4]<bounds[2])
subchain2 = sampler.flatchain[cond]

cond = np.bitwise_and(sampler.flatchain[:,4]>bounds[2], sampler.flatchain[:,4]<bounds[3])
subchain3 = sampler.flatchain[cond]

cond = sampler.flatchain[:,4]>bounds[3]
subchain4 = sampler.flatchain[cond]

In [ ]:
params = [r"$M_{tot}$",
          r"$q$",
          r"$r_p$",
          r"$\delta e$",
          r"$t_\star$",
          r"$f_\star$",]

corner_kwargs = {'bins':30,
                 'labels':params,
                 'label_kwargs':{'fontsize':18},
                 'show_titles':True,
                 'smooth':0.5,
                 'plot_datapoints':False,
                 'plot_density':True,
                 'plot_contours':True,
                 'fill_contours':False,}

corner(subchain4, **corner_kwargs);

In [ ]:
# subchain 1
Mtot = 31
q = 0.63
rpstar = 3.72
destar = 0.31
tstar = -0.056 *(Mtot*Tsun)
fstar = 225 /(Mtot*Tsun)
the_prior(tf_correct_SI, Mtot, q, rpstar, destar, tstar, fstar)

In [ ]:
# subchain 2
Mtot = 31
q = 0.25
rpstar = 3.3
destar = 0.33
tstar = -0.043
fstar = 170
the_prior(tf_correct_SI, Mtot, q, rpstar, destar, tstar, fstar)

In [ ]:
# subchain 4
Mtot = 22
q = 0.10
rpstar = 5
destar = 0.31
tstar = 0
fstar = 170
the_prior(tf_correct_SI, Mtot, q, rpstar, destar, tstar, fstar)

In [ ]:
Nf = 100
Nt = 300

Fs = np.logspace(1, 3, Nf)
Ts = np.linspace(ts[0], ts[-1], Nt)

prior_map = np.zeros([Nf, Nt])

for ii,tt in enumerate(Ts):
    for jj,ff in enumerate(Fs):
        prior_map[jj,ii] = the_prior([[tt,ff]], Mtot, q, rpstar, destar, tstar, fstar)

In [ ]:
t_actual, f_actual = np.array(tf_correct_SI).T

fig = plt.figure(figsize=(8,4.8))
ax = fig.add_subplot(111)
ax.pcolormesh(Ts, Fs, prior_map, cmap='Reds')
ax.scatter(t_actual, f_actual, marker='x', s=150, c='navy')
ax.set_ylim([10, 1000]);
ax.set_xlabel('$t$ (sec)', fontsize=20);
ax.set_ylabel('$f$ (Hz)', fontsize=20);
ax.tick_params(labelsize=16)
ax.set_yscale('log')